# Merging EB terms-  NLS -  Encyclopaedia Britannica


### Loading the necessary libraries

In [1]:
import yaml
import numpy as np
import collections
import string
import copy

In [2]:
import pandas as pd
from yaml import safe_load
from pandas.io.json import json_normalize
from difflib import SequenceMatcher

### Functions

In [3]:
def read_query_results(filename):
    with open('./results_NLS/'+filename, 'r') as f:
        query_results = safe_load(f)
    return query_results


In [4]:
def write_query_results(filename, results):
    with open('./results_NLS/'+filename, 'w') as f:
        documents = yaml.dump(results, f)

In [143]:
def create_dataframe(query_results):

    for edition in query_results:
        for page in query_results[edition]:
            #print(page[1].keys())
            column_list=list(page[1].keys())
            break
        break
        
    data=[]
    for edition in query_results:
        for page in query_results[edition]:
            try:
                data.append(page[1])
               
            except:
                pass
    df = pd.DataFrame(data, columns = column_list)
    #removing the columns that I dont need 
    df= df.drop(['last_term_in_page', 'model', 'num_articles', 'num_page_words', 'num_text_unit' , 'text_unit', 'type_archive'], axis=1)
    #renaming the page num
    df= df.rename(columns={"text_unit_id": "start_page", "type_page": "type_article"})
    
    #removing 'Page' from the string
    df["start_page"] = df["start_page"].str.replace("Page", "")
    df["start_page"] = df["start_page"].astype(int)
    df["end_page"] = df["end_page"].astype(int)
    
    df['term'] = df["term"].str.replace("_def", "")
    df['term']= df["term"].str.replace('[^a-zA-Z0-9]', '')
    
    #mask=df["term"].str.isalpha()
    #df_new=df.loc[mask]
    
    
    
    df['term'] = df['term'].str.upper()
    
    
    df["volume"] = 0
    df["letters"] = ""
    df["part"] = 0
    
    
    mask = df["edition"].str.contains('Volume')
    for i in range(0, len (mask)):
     
        if mask[i]:
            tmp=df.loc[i,'edition'].split("Volume ")[1].split(",")
            if len(tmp)>=1:
                volume= tmp[0]
                letters = tmp[-1].replace(" ","")
                part_tmp = volume.split("Part ")
                if len(part_tmp)>1:
                    volume=part_tmp[0]
                    part = part_tmp[1]
    
                    try:
                        part = int(part)
                    except:
                        if "I" in part:
                            part = 1
                else:
                    part=0

                volume = int(volume)
                df.loc[i, "letters"] = letters
                df.loc[i,"part"] = part
                df.loc[i ,"volume"] = volume
    
    df["edition_num"] = "0"
    list_editions={"1":["first", "First"], "2":["second", "Second"],"3":["third", "Third"],
                   "4":["fourth", "Fourth"], "5":["fifth","Fifth"], "6":["sixth","Sixth"], 
                   "7":["seventh", "Seventh"], "8":["eighth", "Eighth"]}
    
    for ed in list_editions:
        for ed_versions in list_editions[ed]:
            mask = df["edition"].str.contains(ed_versions)
            df.loc[mask, 'edition_num'] = ed
            
            
    df['edition_num']=df["edition_num"].astype(int)    
    df["supplement"]=""
    df["supplement_to_editions"]=""
    
    mask= df["title"].str.contains("Supplement")
    for i in range(0, len (mask)):
        if mask[i]:
            df.loc[i, 'supplement'] = df.loc[i, 'title'] + ","+df.loc[i, 'edition']
            title= df.loc[i, 'title']
            related_editions=[]
            for ed in list_editions:
                for ed_versions in list_editions[ed]:
                    if ed_versions in title:
                        related_editions.append(ed)
                        
            df.loc[i, "supplement_to_editions"]=','.join(related_editions)
            
    
    a=df["archive_filename"].str.split("/").str[-1]
    df['source_text_file']= a+ "/" + df["source_text_file"]   
    df= df.drop(['edition', 'archive_filename'], axis=1)
    
    
    df = df[["term", "definition", "related_terms", "num_article_words", "header", "start_page", "end_page",  "term_id_in_page", "type_article", "edition_num", "supplement", "supplement_to_editions", "volume", "letters", "part",  "year", "title",  "place", "source_text_file"  ]]
    
    df = df[df['term'] != '']
    mask=df["term"].str.isalpha()
    df=df.loc[mask] 
    
    return df

In [6]:
def similar(a, b):
    a=a.lower()
    b=b.lower()
    return SequenceMatcher(None, a, b).ratio()

In [7]:
def most_frequent(List):
    return max(set(List), key = List.count)

In [8]:
def check_string(term, List):
    flag = 0
    for element in List:
        if element in term:
            flag = 1
            break
    if flag == 1:
        return True
    else:
        return False


In [9]:
def clean_topics_terms(term):
    table = str.maketrans('', '', string.ascii_lowercase)
    return term.translate(table)

In [10]:
def create_dataframe_from_file(filename):
    with open('./results_NLS/'+filename, 'r') as f:
        query_results = safe_load(f)
    
    df = create_dataframe(query_results)
    return df

In [11]:
def prune_json(json_dict):
    """
    Method that given a JSON object, removes all its empty fields.
    This method simplifies the resultant JSON.
    :param json_dict input JSON file to prune
    :return JSON file removing empty values
    """
    final_dict = {}
    if not (isinstance(json_dict, dict)):
        # Ensure the element provided is a dict
        return json_dict
    else:
        for a, b in json_dict.items():
            if b or isinstance(b, bool):
                if isinstance(b, dict):
                    aux_dict = prune_json(b)
                    if aux_dict:  # Remove empty dicts
                        final_dict[a] = aux_dict
                elif isinstance(b, list):
                    aux_list = list(filter(None, [prune_json(i) for i in b]))
                    if len(aux_list) > 0:  # Remove empty lists
                        final_dict[a] = aux_list
                else:
                    final_dict[a] = b
    return final_dict

In [12]:
def delete_entries(query_results_updated, eliminate_pages):
    new_results={}
    for edition in query_results_updated:
        new_results[edition]=[]
        for page_idx in range(0, len(query_results_updated[edition])):
            if page_idx not in eliminate_pages[edition]:
                new_results[edition].append(query_results_updated[edition][page_idx])
    return new_results

In [13]:
def deleting_adding_entries(query_results_up, eliminate_pages, create_entries):
    new_results={}
    flag = 1
    for edition in query_results_up:
        new_results[edition]=[]
        for page_idx in range(0, len(query_results_up[edition])):
            if page_idx not in eliminate_pages[edition]:
                new_results[edition].append(query_results_up[edition][page_idx])
            else:
                for new_pages in create_entries[edition][page_idx]:
                    new_results[edition].append(new_pages)
            
        
    return new_results      

In [14]:
def related_terms_info(related_terms):
    
    related_data=[]
    for elem in related_terms:
        if elem.isupper() or "." in elem or "," in elem:
            elem=elem.split(".")[0]
            term=elem.split(",")[0]
            if len(term)>2 and term[0].isupper() :
                m = re.search('^([0-9]+)|([IVXLCM]+)\\.?$', term)
                if m is None:
                    term_up = term.upper()
                    if term_up !="FIG" and term_up !="NUMBER" and term_up!="EXAMPLE" and term_up!="PLATE" and term_up!="FIGURE":
                        related_data.append(term_up) and term_up!="EXAMPLE" and term_up!="PLATE" and term_up!="FIGURE"
                        related_data.append(term_up)
    return related_data

In [15]:
def fixing_articles(query_results):
    
    create_entries={}
    eliminate_pages={}
    for edition in query_results:
        create_entries[edition]={}
        eliminate_pages[edition]=[]
        flag_p = 1
        for page_idx in range(0, len(query_results[edition])):
            element = query_results[edition][page_idx][1]
            element_page = query_results[edition][page_idx][0]
            flag = 0
            
            if element["type_page"]=="Topic" and len(element["term"])<=7:
                
                list_terms=[]
                new_entries=[]
                definition=element["definition"]
                definition_list= definition.split(" ")
                term = element["term"]
                flag = 0
                sub_elements=[]
                for word_idx in range(0, len(definition_list)):
                    word = definition_list[word_idx]
                    if word.isupper() and "," in word and len(word)>3 and "See "!= definition_list[word_idx-1] and "SEE " != definition_list[word_idx-1]:
                        sub_elements.append((word.split(",")[0],word_idx))
                        flag = 1
                        
                     
                if flag and len(sub_elements) >= 5:
                    for elem_idx in range(0, len(sub_elements)):
                        term_id = 0
                       
                        new_element={}
                        elem=sub_elements[elem_idx]
                        new_element["term"]=elem[0]
     
                        if elem_idx+1 < len(sub_elements):
                            sentence=definition_list[elem[1]+1: sub_elements[elem_idx+1][1]]
                            new_element["definition"]=' '.join(sentence)
                       
                            
                        else:
                            new_element["last_term_in_page"] = 1
                            try:
                                sentence= definition_list[elem[1]+1:][1]
                                new_element["definition"]=' '.join(sentence)
        
                            except:
                                sentence= definition_list[elem[1]:]
                                if len(sentence) > 3:
                                    new_element["definition"]=' '.join(sentence)
       
                        if "definition" in new_element and len(new_element["term"])>4:
                            
                            new_element["type_page"] = "Article" 
                            new_element["num_article_words"] = len(sentence)  
                            #### related terms ##### 
                            related_terms=[]
                            if "See " in new_element["definition"]:
                                related_terms= new_element["definition"].split("See ")[1]
                            elif "SEE " in new_element["definition"]:
                                related_terms= new_element["definition"].split("SEE ")[1]  
                            new_element["related_terms"]=related_terms_info(related_terms)
                            ####
                            
                            new_element["term_id_in_page"]=term_id 
                            new_element["archive_filename"]= element["archive_filename"]
                            new_element["header"] = element["header"]
                            new_element["model"] = element["model"]
                            new_element["num_page_words"]= element["num_page_words"]
                            new_element["num_text_unit"] = element["num_text_unit"]
                            new_element["place"] = element["place"]
                            new_element["source_text_file"] = element["source_text_file"]
                            new_element["text_unit"] = element["text_unit"]
                            new_element["text_unit_id"] = element["text_unit_id"]
                            new_element["title"] = element["title"]
                            new_element["type_archive"] = element["type_archive"]
                            new_element["year"] = element["year"]
                            new_element["end_page"] =int(element['text_unit_id'].split("Page")[1])
                            new_element["edition"] = element["edition"]
                            
                            new_entries.append(new_element)
                            list_terms.append(new_element["term"])
                            term_id += 1
                        
                if len(list_terms) > 12:
                    for i in new_entries:
                        i["num_articles"] = len(list_terms)
                    eliminate_pages[edition].append(page_idx)
                    create_entries[edition][page_idx]=[]
                    for new_d in new_entries:
                        create_entries[edition][page_idx].append([element_page, new_d])
                             
   
    new_results = deleting_adding_entries(query_results, eliminate_pages, create_entries)
    return new_results
 

In [16]:
def fixing_topics(query_results):
     for edition in query_results:
        for page_idx in range(0, len(query_results[edition])):
            element = query_results[edition][page_idx][1]
            if (element["num_articles"] == 1) and ((element["type_page"]=="Article") or element["type_page"]=="Mix"):
                prev_element = query_results[edition][page_idx-1][1]
                element_term = element["term"]
                if prev_element["type_page"]=="Topic":
                    element["type_page"]=="Topic"
                    element["term"]=prev_element["term"]
                    print("Moved %s to %s" %(element_term, prev_element["term"] ))
    

In [17]:
def merge_articles(query_results):
    eliminate_pages={}
    for edition in query_results:
        eliminate_pages[edition]=[]
        page_number_dict={}
        for page_idx in range(0, len(query_results[edition])):
            
            current_page=query_results[edition][page_idx][0]
            
            if current_page not in page_number_dict:
                page_number_dict[current_page]=page_idx
            
            element = query_results[edition][page_idx][1]
            
            ### checking the first 20 pages and transforming them to FullPages #### 
            if int(current_page) < 20:
                if element["type_page"]!="FullPage":
                    element = page2full_pages(element)
            
                next_element= query_results[edition][page_idx+1][1]
                if element["type_page"]!="FullPage" and next_element["type_page"]=="FullPage":
                    element["type_page"] = "FullPage"
            
            ###########################################        
            
            if "previous_page" in element['term']:
                current_definition= element["definition"]
                previous_page_idx= page_idx -1
                previous_page_number = current_page -1
                num_article_words=element["num_article_words"]
                related_terms=element["related_terms"]
            
                
                prev_elements = query_results[edition][previous_page_idx][1]
                if prev_elements["last_term_in_page"]:
                   
                    prev_elements["definition"]+=current_definition
                    prev_elements["num_article_words"]+=num_article_words
                    prev_elements["related_terms"]+= related_terms
                    prev_number = int(prev_elements['text_unit_id'].split("Page")[1])
                    prev_elements["end_page"] = current_page
                    
                    for prev_articles_idx in range(page_number_dict[prev_number], page_idx):
                       
                        if query_results[edition][prev_articles_idx][0] == prev_number:
                           
                            query_results[edition][prev_articles_idx][1]["num_page_words"]+=num_article_words
                    
                  
                    for update_element_idx in range(page_number_dict[current_page], page_idx+1):
                        if query_results[edition][update_element_idx][0] == current_page:
                            query_results[edition][update_element_idx][1]["num_page_words"]-=num_article_words
                            query_results[edition][update_element_idx][1]["num_articles"]-=1
                    
                
                eliminate_pages[edition].append(page_idx)
            else:
                element["end_page"] = current_page  
   
    new_results= delete_entries(query_results, eliminate_pages)
    
    return new_results

In [18]:
def merge_topics(query_results):
    eliminate_pages={}
    provenance_removal={}
    freq_topics_terms={}
    merged_topics={}
    parts_string=["Part", "Fart", "Parc", "CPart", "PI"]
    for edition in query_results:
        eliminate_pages[edition]=[]
        provenance_removal[edition]=[]
        freq_topics_terms[edition]={}
        merged_topics[edition]={}
        
        page_idx = 0
        while page_idx < len(query_results[edition]):
            current_page=query_results[edition][page_idx][0]        
            element = query_results[edition][page_idx][1]

            if "Topic" in element['type_page'] and element["term"]!="" and element["term"]!=" ":
                
                if check_string(element["term"], parts_string):
                   
                    ### It means that the previous page was a topic
                    ### And we have one before and correct the error
                    page_idx = page_idx -1 
                    element = query_results[edition][page_idx][1]
                    element['type_page']="Topic"
                    
            
                term=element["term"]
                clean_term=clean_topics_terms(term)
                
                next_page_idx= page_idx + 1
                      
                if next_page_idx < len(query_results[edition]):
                    flag=0
                    tmp_idx = 0
                    for p_id in range(next_page_idx, len(query_results[edition])):
                        next_element = query_results[edition][p_id][1]
   
                        if not check_string(next_element["term"], parts_string):
                            next_term=clean_topics_terms(next_element["term"])
                        else:
                            next_term=next_element["term"]
                    
                        if similar(clean_term, next_term) > 0.72 or check_string(next_term, parts_string) or next_term in clean_term: 

                            if term not in merged_topics[edition]:
                                merged_topics[edition][clean_term]=[]
                        
                            if not check_string(next_term, parts_string) :
                                 merged_topics[edition][clean_term].append(next_term)
                         
                            element["definition"]+=next_element["definition"]
                            element["num_article_words"]+=next_element["num_article_words"]
                            element["num_page_words"]+=next_element["num_page_words"]                  
                            element["related_terms"]+= next_element["related_terms"]
                            element["end_page"] = int(next_element['text_unit_id'].split("Page")[1])
                            provenance_removal[edition].append(element["end_page"])

                            eliminate_pages[edition].append(p_id)
                            tmp_idx= p_id + 1
                        else:
                            flag = 1
                            break
                  
        
                    if flag:
                         page_idx= p_id
                    
                    else:
                        page_idx = tmp_idx
                        
                 
                    if clean_term in merged_topics[edition]:
                       
                        if merged_topics[edition][clean_term]:
                            freq_term=most_frequent(merged_topics[edition][clean_term])
                            freq_topics_terms[edition][term]=freq_term
                            element["term"]=freq_term
                        else:
                            element["term"]=clean_term
                    else:
                        element["term"]=clean_term
                    
                else:
                    page_idx = next_page_idx
                    
                
               
            else:
                page_idx += 1
           
    #for ed in provenance_removal:
    #    print("ED:%s -- removing the following pages %s" %(ed, provenance_removal[ed]))
    new_results= delete_entries(query_results, eliminate_pages)
    
    return new_results, merged_topics, freq_topics_terms

In [19]:
def merge_topics_refine(query_results):
    
    topics_editions={}
    eliminate_pages={}
    merged_topics_refine={}
    provenance_removal={}
    for edition in query_results:
        eliminate_pages[edition]=[]
        provenance_removal[edition]=[]
        topics_editions[edition]={}
        merged_topics_refine[edition]=[]
        page_idx = 0
        character="A"
        while page_idx < len(query_results[edition]):
            
            element = query_results[edition][page_idx][1]
            term = element["term"]
           
            
            if "Topic" in element['type_page'] and term!="" and term!=" " and term[0] >= character:
                character=term[0]
                if term not in topics_editions[edition]:
                    topics_editions[edition][term]=page_idx
                    #print("NEW: Topic --%s-- - Page: %s "%(term, topics_editions[edition][term]))
                else:
                    start_page_idx=topics_editions[edition][term]
                    first_element = query_results[edition][start_page_idx][1]
                    
                    first_element["definition"]+=element["definition"]
                    first_element["num_article_words"]+=element["num_article_words"]
                    first_element["num_page_words"]+=element["num_page_words"]                  
                    first_element["related_terms"]+= element["related_terms"]
                    first_element["end_page"] = int(element['text_unit_id'].split("Page")[1])
                    provenance_removal[edition].append(first_element["end_page"])
                    merged_topics_refine[edition].append(term)
                    eliminate_pages[edition].append(page_idx)
                    
                
            
            page_idx += 1
            if term:
                character=term[0]
        
    new_results= delete_entries(query_results, eliminate_pages)
    
    return new_results, provenance_removal, merged_topics_refine

In [20]:
def page2full_pages(element):
    


    term = element["term"]
    header = element["header"]
    type_page = element["type_page"]
    
    if ("PREFACE" in term) or ("PREFACE" in header):
        term = "PREFACE"
        header = "PREFACE"
        type_page="FullPage"
    
    elif ("Plate" in term) or ("Plafr" in term) or ("Elate" in term) or ("Tlafe" in term):
        header = "Plate"
        term = "Plate"
        type_page = "FullPage"
        
    elif ("Plate" in header) or ("Plafr" in header) or ("Elate" in header) or ("Tlafe" in header):
        header = "Plate"
        term = "Plate"
        type_page = "FullPage"
        
    elif ("ARTSandSCI" in term) or ("ARTSandSCI" in header):
        header = "FrontPage"
        term = "FrontPage"
        type_page="FullPage"

        
    elif "ERRATA" in term or ("ERRATA" in header):
        header = "ERRATA"
        term = "ERRATA"
        type_page="FullPage"
        
   
    elif (" LISTofAUTHORSc" in term) or ("LISTofAUTHORS" in term) or ("ListofAUTHORS" in term) or ("listofAuthors" in term) or ("ListOfAuthors" in term) or ("listofauthors" in term):
        header = "AuthorList"
        term = "AuthorList"
        type_page="FullPage"
        
    elif ("LISTofAUTHORSc" in header) or ("LISTofAUTHORS" in header) or ("ListofAUTHORS" in header) or ("listofAuthors" in header) or ("ListOfAuthors" in header) or ("listofauthors" in header):
        header = "AuthorList"
        term = "AuthorList"
        type_page="FullPage"
        
       
    
    element["term"] = term
    element["header"] = header
    element["type_page"] = type_page
    return element



### 1. Reading data

Here we are going to take the output of the defoe files, and we are going to merge the terms that splitted across pages. 

The next line takes time!

In [21]:
query_results=read_query_results('results_eb_1_edition')

In [22]:
dc_results = copy.deepcopy(query_results)

### 2. Merging articles splitted across pages

Articles normaly have a lenght of one or two paragraphs, but they could be splitted across two consecutive pages.
So, we are going to merge the detected splitted articles into one. 

**NEW** Furthemore, we are going to check the first 20 elements (pages), and check again if they should be FullPage or not based on their header and terms. 

In [23]:
query_results_articles = merge_articles(dc_results)

Since some pages with articles could have wronlgy been classified as "topics", we are going to go through the "TOPICS" pages, and try to extract again articles from those again - reclassifying those as Articles. 

In [24]:
articles_refined=fixing_articles(query_results_articles)

### 3. Merging topics together across pages 

First, we are going to transfor those pages, which only have 1 article, and the previous term is a topic, into topics, and give the previous term to these ones.

In [25]:
fixing_topics(articles_refined)

Moved first_article to ALGEBRA
Moved first_article to ANATOMY
Moved first_article to RITHMETICK
Moved first_article to ARITHMETICK
Moved first_article to WASTE-BOOK
Moved first_article to BOOKKEEPING
Moved first_article to BOOK-KEEPING
Moved first_article to METAPHYSICS
Moved first_article to ilate
Moved first_article to aPNEUMATICS
Moved first_article to SHORTHANDWRITING
Moved first_article to ARITHMETICK
Moved 31S to ARITHMARITHM
Moved first_article to oARITHM
Moved first_article to BOOKKEEPINg
Moved first_article to BOOKKEEPING
Moved first_article to BOOKK
Moved first_article to BOOKKEEPING
Moved first_article to BREWNG
Moved first_article to CHEMstry
Moved first_article to CHEMISTRY
Moved COMMERCE to andthefeedsareovatedspongyandpersistent
Moved first_article to FAIIRIERY
Moved first_article to MEDICINE
Moved first_article to MEDICINE
Moved first_article to MEDICINE
Moved first_article to MIDWIFERY
Moved ’I to MUSICK
Moved first_article to NAVIOATION


Now, we are goint to merge topics.
Topics can be across several pages, so we are going to **merge consecutive page-topics together**.


In [26]:
topics_refined, merged_topics, freq_topics_terms =merge_topics(articles_refined)

In [27]:
#for ed in merged_topics:
#    print("ED: %s, meged_topics %s" %(ed, merged_topics[ed]))

In [28]:
#for ed in freq_topics_terms:
#    print("ED: %s, freq_topics_terms %s" %(ed, freq_topics_terms[ed]))

#### 3.1 Merging topics even more

Before, we were merging topics that were in consecutive pages. But very often we have other types of pages (with tables, figures) between the same topic. Therefore we are going to merge the pages with the same topics. 

In [29]:
final_refine, provenance_removal,merged_topics_refine =merge_topics_refine(topics_refined)

In [30]:
for ed in merged_topics_refine:
    print("ED: %s, merged_topics_refine %s" %(ed, merged_topics_refine[ed]))

ED: First edition, 1771, Volume 1, A-B, merged_topics_refine ['ALGEBRA', 'ALGEBRA', 'ANATOMY', 'ARCHITECTURE', 'ARCHITECTURE', 'ARITHMETICK', 'ARITHMETICK', 'ASTRONOMY', 'ASTRONOMY', 'BOTANY', 'BOTANY']
ED: First edition, 1771, Volume 2, C-L, merged_topics_refine ['CHEMISTRY', 'CHEMISTRY', 'FARRIERY', 'SFLUXIONS', 'GEOMETRY', 'GEOMETRY', 'GRAMMAR', 'GRAMMAR', 'LA', 'LA', 'LLW', 'LAW', 'LAW', 'LA', 'LAW', 'LAW', 'LAW']
ED: First edition, 1771, Volume 3, M-Z, merged_topics_refine ['MEDICINE', 'METAPHYSICS', 'METAPHYSICS', 'MIDWIFERY', 'MIDWIFERY', 'MIDWIFERY', 'NAVIGATION', 'OPTICS', 'I', 'OPTICS', 'SHORTHANDWRITING', 'SURGERY', 'SURGERY', 'SURGERY', 'SURGERY', 'SURGEY', 'SURGERY', 'TRIGONOMETRY', 'TRIGONOMETRY']
ED: First edition, 1773, Volume 1, A-B, merged_topics_refine ['ARITHMETICK', 'ARITHMETICS', 'ARITHM', 'ARITHMETICK', 'P']
ED: First edition, 1773, Volume 2, C-L, merged_topics_refine ['CHEMISTRY', 'CHEMICHEMI', 'CHEMISTRY', 'FARRIERY', 'FARRIERY', 'FLUXIONS', 'GARDENING', 'GARDE

Once the data has been merged, we are going to store it in a file, just to have the data merged.  

In [31]:
write_query_results("results_eb_1_edition_updated", final_refine)

### 2. Creating a dataframe from the updated results

Once, we have the terms properly merged, we are going to create a dataframe, which we will be use later to do further exploration. In this dataframe we have dropped some information from the original defoe files, that we don not longer need. 

**The dataframe will have the following columns**

- definition:           Definition of the article
- edition_num:          1,2,3,4,5,6,7,8
- header:               Header of the page's article                                  
- num_article_words:    Number of words per article
- place:                Place where the volume was edited (e.g. Edinburgh)                                    
- related_terms:        Related articles (see X article)  
- source_text_file:     File Path of the XML file from which the article belongs       
- term:                 Article name                            
- term_id_in_page:      Number of article in the page     
- start_page:           Number page in which the article starts 
- end_page:             Number page in which the article ends 
- title:               Title of the Volume
- type_article:            Type of Page [Full Page| Topic| Mix | Articles]                                       
- year:                 Year of the Volume
- volume:               volume (e.g. 1)
- letters:              leters of the volume (A-B)
- part:                 part of the volume (e.g 1)


### IMPORTANT DECISION

I am going to filter OUT all the entries which are not Articles, Topics, or Mix.

In [144]:
#df=create_dataframe_from_file("results_eb_1_edition_updated")

In [145]:
for i in final_refine:
    print(final_refine[i][0][1]["edition"])
    break

First edition, 1771, Volume 1, A-B


In [146]:
df=create_dataframe(final_refine)

In [147]:
includeKeywords=["Article", "Topic", "Mix"]
df=df[df["type_article"].str.contains('|'.join(includeKeywords))]

In [148]:
df.head(n=20)

,term,definition,related_terms,num_article_words,header,start_page,end_page,term_id_in_page,type_article,edition_num,supplement,supplement_to_editions,volume,letters,part,year,title,place,source_text_file
7,OR,"A NEW A D I C T I A A, the name of several riv...",[],54,EncyclopaediaBritannica,15,15,0,Article,1,,,1,A-B,0,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188082904.34.xml
8,AABAM,"a term, among alchemifts, for lead,",[],6,EncyclopaediaBritannica,15,15,1,Article,1,,,1,A-B,0,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188082904.34.xml
9,AACH,the name of a town and river in Swabia. It is ...,[],17,EncyclopaediaBritannica,15,15,2,Article,1,,,1,A-B,0,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188082904.34.xml
10,AADE,"the name of two rivers, one in the country of ...",[],19,EncyclopaediaBritannica,15,15,3,Article,1,,,1,A-B,0,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188082904.34.xml
11,AAHUS,a small town and diftrift in Weftphalia.,[],7,EncyclopaediaBritannica,15,15,4,Article,1,,,1,A-B,0,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188082904.34.xml
12,AAM,"a Dutch measure for liquids, containing about ...",[],17,EncyclopaediaBritannica,15,15,5,Article,1,,,1,A-B,0,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188082904.34.xml
13,AAR,"the name of two rivers, one in Weftphalia, and...",[],25,EncyclopaediaBritannica,15,15,6,Article,1,,,1,A-B,0,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188082904.34.xml
14,AARSEO,"a town in Africa, situated near the mouth of",[],9,EncyclopaediaBritannica,15,15,7,Article,1,,,1,A-B,0,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188082904.34.xml
15,AATTER,"or AT TER, a province of Arabia Felix, situate...",[],36,EncyclopaediaBritannica,15,15,8,Article,1,,,1,A-B,0,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188082904.34.xml
16,AB,the eleventh month of the civil year of the He...,[],136,EncyclopaediaBritannica,15,15,9,Article,1,,,1,A-B,0,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188082904.34.xml


In [149]:
df.columns

Index(['term', 'definition', 'related_terms', 'num_article_words', 'header',
       'start_page', 'end_page', 'term_id_in_page', 'type_article',
       'edition_num', 'supplement', 'supplement_to_editions', 'volume',
       'letters', 'part', 'year', 'title', 'place', 'source_text_file'],
      dtype='object')

### 3. Saving the dataframe to json file 

In [150]:
df.to_json(r'./results_NLS/results_eb_1_edition_updated_postprocess_dataframe', orient="index") 

In [151]:
df[df["term"]=="FIRSTARTICLE"]


,term,definition,related_terms,num_article_words,header,start_page,end_page,term_id_in_page,type_article,edition_num,supplement,supplement_to_editions,volume,letters,part,year,title,place,source_text_file
1736,FIRSTARTICLE,N N U I T I E S.,[],8,(318),368,368,0,Mix,1,,,1,A-B,0,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188087496.34.xml
3397,FIRSTARTICLE,"O O K K terms Debtor and Creditor, because the...",[],528,BOOKK,695,695,0,Mix,1,,,1,A-B,0,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188091747.34.xml
3404,FIRSTARTICLE,O O K W A S T E B O O K.,[],12,SBOOK,716,716,0,Article,1,,,1,A-B,0,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188092020.34.xml
3926,FIRSTARTICLE,"lat. and the latter, thirty miles north of Lin...",[],86,BURBUS,826,826,0,Article,1,,,1,A-B,0,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188093450.34.xml
3968,FIRSTARTICLE,I C T I,[],4,EncyclopaediaBritannica,9,9,0,Article,1,,,2,C-L,0,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133902/alto/144913710.34.xml
4705,FIRSTARTICLE,H E M ing the vest:l. This danger is easily av...,[],941,HCHEM,141,141,0,Article,1,,,2,C-L,0,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133902/alto/144915318.34.xml
4707,FIRSTARTICLE,H E M I of phloglfton which conftitutes it ste...,[],734,YSTR,159,159,0,Article,1,,,2,C-L,0,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133902/alto/144915534.34.xml
7076,FIRSTARTICLE,fl that can l>e done is to empty the vefle Js ...,[],46,FAHHT,503,503,0,Article,1,,,2,C-L,0,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133902/alto/144919662.34.xml
7329,FIRSTARTICLE,O R T I F I,[],6,(«'7),579,579,0,Mix,1,,,2,C-L,0,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133902/alto/144920574.34.xml
8027,FIRSTARTICLE,"EPATUS, in ichthyology. See Labrus. HEP Hi EST...","[LABRUS, HEP, ESTIA, HVAL, VULCAN, HEPSETUS]",34,HERHER,777,777,0,Article,1,,,2,C-L,0,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133902/alto/144922950.34.xml


In [152]:
df[df["term"]=="ABERDEEN"]

,term,definition,related_terms,num_article_words,header,start_page,end_page,term_id_in_page,type_article,edition_num,supplement,supplement_to_editions,volume,letters,part,year,title,place,source_text_file
91,ABERDEEN,"the name of two cities in Scotland, called the...",[],415,ABE,18,18,0,Article,1,,,1,A-B,0,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188082943.34.xml
13902,ABERDEEN,"the name of two cities in Scotland, called the...",[],413,ABEABI,18,18,1,Article,1,,,1,A-B,0,1773,"Encyclopaedia Britannica: or, A dictionary of ...",London,144850366/alto/188283831.34.xml


In [153]:
df[df['term'].str.contains("ABACUS")]

,term,definition,related_terms,num_article_words,header,start_page,end_page,term_id_in_page,type_article,edition_num,supplement,supplement_to_editions,volume,letters,part,year,title,place,source_text_file
27,ABACUS,"a table strewed over with dust or sand, upon w...",[],23,EncyclopaediaBritannica,15,15,20,Article,1,,,1,A-B,0,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188082904.34.xml
28,ABACUS,"in architeflure, signifies the superior part o...",[],122,EncyclopaediaBritannica,15,16,21,Article,1,,,1,A-B,0,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188082904.34.xml
29,ABACUS,is also the name of an ancient instrument for ...,[],125,ABAABB,16,16,1,Article,1,,,1,A-B,0,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188082917.34.xml
30,ABACUS,"logijlicus, a right-angled triangle, whose sid...",[],50,ABAABB,16,16,2,Article,1,,,1,A-B,0,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188082917.34.xml
13840,ABACUS,is also the name of an ancient inllrument for ...,[],127,ABAABB,16,16,1,Article,1,,,1,A-B,0,1773,"Encyclopaedia Britannica: or, A dictionary of ...",London,144850366/alto/188283805.34.xml
13841,ABACUS,"Pythagoricus, the multiplication-table, or any...",[],13,ABAABB,16,16,2,Article,1,,,1,A-B,0,1773,"Encyclopaedia Britannica: or, A dictionary of ...",London,144850366/alto/188283805.34.xml
